## 2.1 Course Info Data Cleaning

### Import libraries

In [1]:
# the basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import preprocessing and modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# immport metrics
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

pd.options.display.max_colwidth = 350

### Data Cleaning: Independent variables (X)
#### Import data

In [3]:
path = '../data/ds_course_df.csv'
course = pd.read_csv(path)

# quick pre-processing, dropping columns and remove rows with null values
course.drop(columns=['Unnamed: 0'], inplace=True)
course.dropna(inplace=True)

In [4]:
course.shape

(560, 11)

In [5]:
course.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   course_href    560 non-null    object 
 1   course_name    560 non-null    object 
 2   partner_title  560 non-null    object 
 3   stars          560 non-null    float64
 4   recent_views   560 non-null    object 
 5   num_ratings    560 non-null    object 
 6   num_reviews    560 non-null    int64  
 7   description    560 non-null    object 
 8   outcome        560 non-null    object 
 9   highlight      560 non-null    object 
 10  length         560 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 52.5+ KB


In [6]:
course.head(1)

,course_href,course_name,partner_title,stars,recent_views,num_ratings,num_reviews,description,outcome,highlight,length
0,/learn/exploratory-data-analysis,Exploratory Data Analysis,Johns Hopkins University,4.7,"108,049 recent views","5,836 ratings",845,This course covers the essential exploratory techniques for summarizing data. These techniques are typically applied before formal modeling commences and can help inform the development of more complex statistical models. Exploratory techniques are also important for eliminating or sharpening potential hypotheses about the world that can be add...,"['38%started a new career after completing these courses', '15%got a pay increase or promotion', '38%got a tangible career benefit from this course']","['100% online', 'Shareable Certificate', 'Flexible deadlines', 'English']",Approx. 55 hours to complete


### Cleaning up the data in each of the column

#### Remove all non-numeric characters `recent_view`, `num_ratings`, and `length` columns

In [8]:
'''
remove all non-numeric characters from a string
''.join(c for c in "abc123def456" if c.isdigit())

https://stackoverflow.com/questions/1249388/removing-all-non-numeric-characters-from-string-in-python
''.join(c for c in '2,103,753 recent views' if c.isdigit())
'''
# remove all non-numeric characters from a string
course['recent_views'] = course['recent_views'].map(lambda x: int((''.join(c for c in x if c.isdigit()))))
course['num_ratings'] = course['num_ratings'].map(lambda x: int((''.join(c for c in x if c.isdigit()))))
course['length'] = course['length'].map(lambda x: int((''.join(c for c in x if c.isdigit()))))

In [9]:
course.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   course_href    560 non-null    object 
 1   course_name    560 non-null    object 
 2   partner_title  560 non-null    object 
 3   stars          560 non-null    float64
 4   recent_views   560 non-null    int64  
 5   num_ratings    560 non-null    int64  
 6   num_reviews    560 non-null    int64  
 7   description    560 non-null    object 
 8   outcome        560 non-null    object 
 9   highlight      560 non-null    object 
 10  length         560 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 52.5+ KB


#### Dummify `highlight` into individual features

In [10]:
# example: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
# using .apply() and .eval() to turn string into list of items
course['highlight'] = course['highlight'].apply(eval)

In [11]:
# this is a function adopted from https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [12]:
list_highlights = list(to_1D(course['highlight']).unique())

In [14]:
# this is a function adopted from https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
def boolean_df(item_lists, unique_items):
# this function return a boolean dataframe with the unique items unpacked from the original column.
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
        
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [15]:
# create the boolean dataframe with only the items in the highlight column
highlight_bool = boolean_df(course['highlight'], list_highlights)

In [17]:
# create a list that include all the highlight items that describe a course is part of a specialization
serials = [col for col in highlight_bool.columns if 'in the' in col] 

In [19]:
# create a new dataframe with only the highlight items that describe a course is part of a specialization
serials_df = highlight_bool[serials]

# sum all columns in one row, and save the result in a new column,
# serials_df['part_of_specialization'] == 1 describe the course is part of a specialization
# sum by row: df["sum"] = df.sum(axis=1)
serials_df['part_of_specialization'] = serials_df.sum(axis=1)
serials_df.head(1)

<ipython-input-19-5b5e554c1ddf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serials_df['part_of_specialization'] = serials_df.sum(axis=1)


,Course 4 of 6 in the,Course 1 of 3 in the,Course 1 of 4 in the,Course 4 of 4 in the,Course 1 of 7 in the,Course 2 of 3 in the,Course 3 of 4 in the,Course 4 of 5 in the,Course 1 of 5 in the,Course 3 of 5 in the,...,Course 4 of 8 in the,Course 9 of 9 in the,Course 5 of 7 in the,Course 4 of 7 in the,Course 6 of 8 in the,Course 3 of 6 in the,Course 1 of 8 in the,Course 2 of 7 in the,Course 7 of 7 in the,part_of_specialization
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0


In [20]:
# create a new column in the highlight data frame to indicate whether a course is part of specialization 
# drop the original itemized columns
highlight_df = highlight_bool.drop(columns=serials)
highlight_df['specialization'] = serials_df['part_of_specialization']

In [21]:
# fix the column names 
highlight_df.columns = highlight_df.columns.str.lower()
highlight_df.columns = highlight_df.columns.str.replace(' ', '_')

In [22]:
for each in highlight_df.columns:
    highlight_df[each] = highlight_df[each].astype(int)

In [23]:
# course.shape
# highlight_df.shape

course = pd.concat([course, highlight_df], axis=1)

#### Clean the `outcome` column

In [25]:
# example: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
# using .apply() and .eval() to turn string into list of items
course['outcome'] = course['outcome'].apply(eval)

In [26]:
# define a function to strip the core content of an outcome

def to_core_outcomes(list_of_outcomes):
    coure_string_dict = {}
    
    for each in list_of_outcomes:
        value = ''.join([c for c in each if c.isdigit()])
        key = each.replace(value, '')
        coure_string_dict[key] = int(value)
    return coure_string_dict

In [27]:
# testing 
to_core_outcomes(course['outcome'][0])

{'%started a new career after completing these courses': 38,
 '%got a pay increase or promotion': 15,
 '%got a tangible career benefit from this course': 38}

In [28]:
course['outcome_core'] = course['outcome'].apply(to_core_outcomes)

In [29]:
course.head(1)

,course_href,course_name,partner_title,stars,recent_views,num_ratings,num_reviews,description,outcome,highlight,...,chinese_(traditional),arabic,portuguese_(brazilian),russian,advanced_level,chinese_(simplified),french,japanese,specialization,outcome_core
0,/learn/exploratory-data-analysis,Exploratory Data Analysis,Johns Hopkins University,4.7,108049,5836,845,This course covers the essential exploratory techniques for summarizing data. These techniques are typically applied before formal modeling commences and can help inform the development of more complex statistical models. Exploratory techniques are also important for eliminating or sharpening potential hypotheses about the world that can be add...,"[38%started a new career after completing these courses, 15%got a pay increase or promotion, 38%got a tangible career benefit from this course]","[100% online, Shareable Certificate, Flexible deadlines, English]",...,0,0,0,0,0,0,0,0,0,"{'%started a new career after completing these courses': 38, '%got a pay increase or promotion': 15, '%got a tangible career benefit from this course': 38}"


In [30]:
list_outcomes = ['%got a tangible career benefit from this course',
 '%got a pay increase or promotion',
 '%started a new career after completing these courses']

In [31]:
def extract_career_benefit(outcome_dict):
    try:
        return outcome_dict['%got a tangible career benefit from this course']
    except:
        return np.nan

In [32]:
def extract_pay_increase(outcome_dict):
    try:
        return outcome_dict['%got a pay increase or promotion']
    except:
        return np.nan

In [33]:
def extract_new_career(outcome_dict):
    try:
        return outcome_dict['%started a new career after completing these courses']
    except:
        return np.nan

In [34]:
# extract_career_benefit(outcome_bool['outcome_dict'][0])

In [35]:
# course['outcome_dict'].apply(extract_career_benefit)

In [36]:
course['outcome_career_benefit'] = course['outcome_core'].apply(extract_career_benefit)

In [37]:
course['outcome_pay_increase'] = course['outcome_core'].apply(extract_pay_increase)

In [38]:
course['outcome_new_career'] = course['outcome_core'].apply(extract_new_career)

In [39]:
course = course.drop(columns=['highlight','outcome', 'outcome_core'])

#### Create dummies for university partners: `he_partner`
**1: indicating the partner is a university/college vs. companies (i.e. IBM, Google)**

In [44]:
def is_university(partner_name):
    if 'university' in partner_name.lower():
        return 1
    elif 'college' in partner_name.lower():
        return 1
    else:
        return 0

In [45]:
course['he_partner'] = course['partner_title'].apply(is_university)

In [46]:
course.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   course_href             560 non-null    object 
 1   course_name             560 non-null    object 
 2   partner_title           560 non-null    object 
 3   stars                   560 non-null    float64
 4   recent_views            560 non-null    int64  
 5   num_ratings             560 non-null    int64  
 6   num_reviews             560 non-null    int64  
 7   description             560 non-null    object 
 8   length                  560 non-null    int64  
 9   100%_online             560 non-null    int64  
 10  shareable_certificate   560 non-null    int64  
 11  flexible_deadlines      560 non-null    int64  
 12  english                 560 non-null    int64  
 13  intermediate_level      560 non-null    int64  
 14  beginner_level          560 non-null    in

### Data Cleaning: Dependent variables (y)
#### Import data 

In [49]:
enrollment = pd.read_csv('../data/ds_enrollment.csv')
enrollment.drop(columns=['Unnamed: 0'], inplace=True)

In [50]:
enrollment.head()

,course_href,enrollment
0,/learn/exploratory-data-analysis,"157,581"
1,/learn/clinical-natural-language-processing,"3,341"
2,/learn/machine-learning-with-python,"199,048"
3,/learn/material-informatics,"11,573"
4,/learn/complete-reinforcement-learning-system,"11,701"


In [51]:
course = course.merge(enrollment,how='inner', on='course_href')

In [52]:
course.drop_duplicates(inplace=True, ignore_index=True)

In [53]:
course['enrollment'] = course['enrollment'].map(lambda x: int((''.join(c for c in x if c.isdigit()))))

In [55]:
course.shape

(348, 30)

In [56]:
# uncomment to run code
# course.to_csv('../data/ds_course_modeling.csv')

In [1]:
# end of codebook